## Feature Request: Integrate AP Grade Predictor into Frontend

### Overview

- **Purpose**: Provide students with grade predictions based on self-assessment.
- **Integration**: Part of the Palomar Health Frontend.
- **Output**: Percentage score and letter grade saved to user profiles.

#### Challenges Faced:
- **Slow Start**: Unclear requirements and technical hurdles delayed progress.
- **Database Issues**: Difficulties updating the user table with prediction data.
- **CORS Errors**: Blocked API requests during frontend integration.

---

### The AI Model

- **Model**: Linear Regression.
- **Dataset**: `ap_predict_data.csv` with 11 input features:
  - Attendance, Work Habits, Behavior, Timeliness, Advocacy, Tech Growth,
    Tech Sense, Tech Talk, Communication and Collaboration, Leadership, Integrity.
- **Preprocessing**:
  - Grades mapped to percentages (A=90, B=80, etc.).
  - Inputs binarized (1–3 → 0, 4–5 → 1).

#### Prediction Output:
- **Percentage Score**: Clamped between 0–100.
- **Letter Grade**:
  - 90+ → A, 80–89 → B, 70–79 → C, 60–69 → D, Below 60 → F.

#### AI Model Workflow:
```
Input Features (11) --> Preprocessing (Binarization) --> Linear Regression Model -->
Predicted Percentage --> Letter Grade Mapping --> Output (Percentage, Letter Grade)
```
---

### Backend API Implementation

- **Framework**: Flask with `flask_restful`.
- **Endpoints**:
  - `/predict` (POST): Accepts user inputs and returns predictions.
  - `/predict` (GET): Retrieves user-specific grade history (JWT-protected).

#### Key Challenges:
- **Database Updates**:
  - Ensuring schema compatibility for new fields.
  - Avoiding data corruption during updates.
- **CORS Errors**:
  - Resolved by adding appropriate headers to Flask responses.

#### Backend Architecture:
```
Frontend --> API Gateway --> Flask App -->
  - Prediction Logic (GradePredictionModel)
  - Database (User Table Updates)
```
---

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

class GradePredictionModel:
    def __init__(self):
        # Load dataset
        data = pd.read_csv("datasets/ap_predict_data.csv")
        data = data.dropna(subset=['Grade'])
        data.columns = data.columns.str.strip()

        # Map grades to numeric percentages
        grade_map = {'A': 90, 'B': 80, 'C': 70, 'D': 60, 'F': 50}
        data['GradePercent'] = data['Grade'].map(grade_map)

        self.features = ['Attendance','Work Habits','Behavior','Timeliness','Advocacy','Tech Growth',
                         'Tech Sense','Tech Talk','Communication and Collaboration','Leadership','Integrity']

        X = data[self.features]
        y = data['GradePercent']

        self.model = LinearRegression()
        self.model.fit(X, y)

    def predict(self, user_input):
        if len(user_input) != len(self.features):
            raise ValueError(f"Expected {len(self.features)} inputs, got {len(user_input)}")

        binarized = [0 if x <= 3 else 1 for x in user_input]

        percent = self.model.predict([binarized])[0]
        percent = max(0, min(100, percent))

        if percent >= 90:
            letter = 'A'
        elif percent >= 80:
            letter = 'B'
        elif percent >= 70:
            letter = 'C'
        elif percent >= 60:
            letter = 'D'
        else:
            letter = 'F'

        return round(percent, 2), letter


In [ ]:
from flask import Blueprint, request, jsonify, g
from flask_restful import Api, Resource
from api.jwt_authorize import token_required
from model.user import User
from model.grade_model import GradePredictionModel

grade_api = Blueprint('grade_api', __name__, url_prefix='/api/grade')
api = Api(grade_api)

model_instance = GradePredictionModel()

class Predict(Resource):
    def post(self):
        data = request.get_json()
        if not data or 'inputs' not in data:
            return {"error": "Missing 'inputs' field in JSON payload. Expected a list of 11 numbers (1-5)."}, 400

        user_input = data['inputs']

        if len(user_input) != 11:
            return {"error": f"Expected 11 input values, received {len(user_input)}."}, 400

        try:
            user_input = [int(val) for val in user_input]
        except ValueError:
            return {"error": "All input values must be numbers between 1 and 5."}, 400

        if not all(1 <= val <= 5 for val in user_input):
            return {"error": "Input values should be between 1 and 5."}, 400

        percent, letter = model_instance.predict(user_input)

        return jsonify({
            'predicted_percent': percent,
            'predicted_grade': letter
        })

    @token_required()
    def get(self):
        user = g.current_user
        return jsonify(user.grade_data)

api.add_resource(Predict, '/predict')

### Lessons Learned

- **Debugging**: Importance of detailed error logs and systematic testing.
- **Cross-Team Communication**: Collaboration between frontend and backend teams was crucial.
- **Thorough Testing**: Prevented regressions and ensured smooth integration.

---

### Final Thoughts

- Despite initial setbacks, the feature is now robust and user-friendly.
- The challenges faced have strengthened our development process.

---